## Process that downloads the historical data of stock market of Technology industry.

#### Imports

In [ ]:
from bs4 import BeautifulSoup
from pandas_datareader import data as pdr
import json
import os
import pandas as pd
import quandl
import requests
import time
import fix_yahoo_finance as yf

#### Global variables and constants

In [ ]:
PATH_PROJECT = os.getcwd()
POSTGRES_DB = 'stock-tracker'
POSTGRES_USER = 'postgres'
POSTGRES_PASS = 'Yj18pQ-$20'
queued_stocks = []

#### 1. Download the symbol list of Technology industry

In [1]:
def technology_stocks():
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'
    }
    try:
        base_url = 'https://api.iextrading.com/1.0/stock/market/collection/sector?collectionName=Technology'
        r = requests.get(base_url, headers = headers)
        if r.status_code == 200:
            src = BeautifulSoup(r.text, 'lxml')
            stock_list = json.loads(src.find('p').text)
            stocks = [stock["symbol"] for stock in stock_list]
            return stocks
        else:
            return None
    except Exception as ex:
        return None

#### 2. Download the historical data of the stock symbols of Technology industry (one by one to ease the upload to DB)

In [ ]:
def historical_report(stock, start_date, end_date):
    yf.pdr_override()
    try:
        data = pdr.get_data_yahoo(
            stock, 
            start = start_date, 
            end = end_date, 
            progress = False
        )
        data['Stock'] = stock
        return data
    except Exception as ex:
        print('Yahoo Finance collapse downloading {stock} data. Adding to queue to continue once the correct downloads end...')
        queued_stocks.append(stock)

#### 3. Upload historical data to PostgreSQL database

In [ ]:
def upload_historical(name):
    conn = None
    commands = (f"""SELECT EXISTS(
            SELECT 1 FROM information_schema.tables
            WHERE table_catalog = {POSTGRES_DB}
                  table_schema = 'public'
                  table_name = {name}
        )""", f"""CREATE TABLE {name} (
            open        double precision NOT NULL,
            high        double precision NOT NULL,
            low         double precision NOT NULL,
            close       double precision NOT NULL,
            adj_close   double precision NOT NULL,
            volume      integer NOT NULL,
            stock       varchar(10) NOT NULL,
            date_d      varchar(10) NOT NULL
        )""")
    try:
        conn = psycopg2.connect(f'dbname={POSTGRES_DB} user={POSTGRES_USER} password={POSTGRES_PASS}')
        cur = conn.cursor()
        cur.execute(commands[0])
        if (cur.fetchone()[0]):
            print('Table already exists.')
        else:
            cur.execute(commands[1])
        
        cur.close()
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as ex:
        print(str(ex))
    finally:
        if conn is not None:
            conn.close()

In [6]:
sssss = "hola"
print(f"Heeeey {sssss}")

Heeeey hola
